In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
campData=pd.read_csv('campaign_data.csv')
coupon_item_mappingData=pd.read_csv('coupon_item_mapping.csv')
customer_demographicsData=pd.read_csv('customer_demographics.csv')
customer_transaction_dataData=pd.read_csv('customer_transaction_data.csv')
item_dataData=pd.read_csv('item_data.csv')
trainData=pd.read_csv('train.csv')
test_QyjYwdjData=pd.read_csv('test_QyjYwdj.csv')

In [3]:
trainData['dtType']='train'
test_QyjYwdjData['dtType']='test'

allData=trainData.append(test_QyjYwdjData,sort=False)
allData.shape

(128595, 6)

In [4]:
customer_transaction_dataData.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [5]:
tX1=pd.merge(customer_transaction_dataData,item_dataData,left_on='item_id',right_on='item_id',how='left')

In [24]:
campaignRespOld=pd.pivot_table(trainData,index='customer_id',columns='campaign_id',values='redemption_status',aggfunc=np.sum).fillna(0).reset_index()
# del campaignRespOld['campaign_id']

In [7]:
trainData.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType
0,1,13,27,1053,0,train
1,2,13,116,48,0,train
2,6,9,635,205,0,train
3,7,13,644,1050,0,train
4,9,8,1017,1489,0,train


In [25]:
customerItemRel=customer_transaction_dataData.groupby('customer_id',as_index=False).agg({'date':{'lastDate':'max','minDate':'min'},
                                                                        'item_id':{'totalCountPurch':'count',
                                                               'uniqueItemPurch':'nunique'}
                                                                    ,'selling_price':{'tPCustomer':'sum','tPCustomerMean':'mean',
                                                                                        'tPCustomerMin':'min'},
                                              'quantity':'sum','other_discount':{'oDCust':'sum','oDCustMean':'mean'},
                                              'coupon_discount':'sum'})

customerItemRel.columns=[''.join(j) for j in customerItemRel.columns ]

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [26]:
import dateutil

In [27]:
customerItemRel['datelastDate']=customerItemRel['datelastDate'].apply(lambda x: dateutil.parser.parse(x))
customerItemRel['dateminDate']=customerItemRel['dateminDate'].apply(lambda x: dateutil.parser.parse(x))
customerItemRel['patron']=customerItemRel['datelastDate']-customerItemRel['dateminDate']
customerItemRel['patron']=customerItemRel['patron'].apply(lambda x: x.days)

In [28]:
del customerItemRel['datelastDate']
del customerItemRel['dateminDate']
customerItemRel.head()

,customer_id,item_idtotalCountPurch,item_iduniqueItemPurch,selling_pricetPCustomer,selling_pricetPCustomerMean,selling_pricetPCustomerMin,quantitysum,other_discountoDCust,other_discountoDCustMean,coupon_discountsum,patron
0,1,1048,463,98513.93,94.001842,12.11,1227,-17030.40,-16.250382,-2116.83,495
1,2,419,352,43100.03,102.864033,14.25,474,-7051.95,-16.830430,-249.34,439
2,3,705,406,73050.27,103.617404,8.90,8163,-16013.53,-22.714227,-2179.54,419
3,4,220,125,33973.22,154.423727,14.25,280,-2927.19,-13.305409,-89.05,392
4,5,792,490,103615.10,130.827146,12.11,93353,-10817.07,-13.657917,-90.83,427


# Things cannot be creaeted from campaign

In [ ]:
# pd.pivot_table(trainData,index='campaign_id',values='redemption_status',aggfunc=np.sum).reset_index()
# trainData.groupby('campaign_id',as_index=False).agg({'redemption_status':{'availed':'sum','totalOccu':'count'}})
# test_QyjYwdjData.groupby('campaign_id',as_index=False).count()[['campaign_id','id']]

In [ ]:
# customerBuyingBehavior=trainData.groupby('customer_id',as_index=False).agg({'redemption_status':{'redeemed':'sum','totalOccur':'count'}})
# customerBuyingBehavior.columns=[''.join(j) for j in customerBuyingBehavior]

In [ ]:
# customer_transaction_dataData.shape,tX1.shape

In [ ]:
# tX1.head()

In [29]:
x1=pd.merge(allData,campData,left_on='campaign_id',right_on='campaign_id',how='left')
x2=pd.merge(x1,customer_demographicsData,left_on='customer_id',right_on='customer_id',how='left')
x2.shape

(128595, 15)

In [30]:
x2['marital_status']=x2['marital_status'].fillna('Other')
x2['rented']=x2['rented'].fillna(0)
x2['no_of_children']=x2['no_of_children'].fillna(0)
x2['age_range']=x2['age_range'].fillna('Other')
x2['family_size']=x2['family_size'].fillna('Other')
x2['income_bracket']=x2['income_bracket'].fillna('Other')

In [31]:
x2.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0.0,train,X,19/05/13,05/07/13,46-55,Other,0.0,1,0,5
1,2,13,116,48,0.0,train,X,19/05/13,05/07/13,36-45,Married,0.0,2,0,3
2,6,9,635,205,0.0,train,Y,11/03/13,12/04/13,46-55,Married,0.0,2,0,7
3,7,13,644,1050,0.0,train,X,19/05/13,05/07/13,Other,Other,0.0,Other,0,Other
4,9,8,1017,1489,0.0,train,X,16/02/13,05/04/13,46-55,Married,0.0,2,0,3


In [32]:
xXX2=pd.merge(x2,customerItemRel,on='customer_id',how='left')
xXX2.shape

(128595, 25)

In [33]:
xXX3=pd.merge(xXX2,campaignRespOld,on='customer_id',how='left')

In [34]:
xXX3.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,campaign_type,start_date,end_date,age_range,...,9,10,11,12,13,26,27,28,29,30
0,1,13,27,1053,0.0,train,X,19/05/13,05/07/13,46-55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,13,116,48,0.0,train,X,19/05/13,05/07/13,36-45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,9,635,205,0.0,train,Y,11/03/13,12/04/13,46-55,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,13,644,1050,0.0,train,X,19/05/13,05/07/13,Other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,8,1017,1489,0.0,train,X,16/02/13,05/04/13,46-55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# customerProfileData=customer_transaction_dataData.groupby('customer_id',as_index=False).agg({'quantity':'sum','date':'nunique','item_id':'count',
#                                                                         'selling_price':{'maxSP':'max','minSP':'min',
#                                                                                         'averageSP':'mean'},
#                                                                         'other_discount':{'maxOD':'max','minOD':'min',
#                                                                                         'averageOD':'mean'},
#                                                                         'coupon_discount':{'maxCD':'max','minCD':'min',
#                                                                                         'averageCD':'mean','totalCDAvailed':'count'}})

In [ ]:
# customerProfileData.columns = ['_'.join(col) for col in customerProfileData.columns.values]

In [36]:
categoColumns=['campaign_type','age_range','marital_status','family_size','no_of_children','income_bracket']
catData=pd.get_dummies(xXX3[categoColumns],prefix_sep='_',)

In [37]:
newtData=pd.concat([xXX3,catData],sort=False,axis=1)

In [38]:
newtData.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,campaign_type,start_date,end_date,age_range,...,income_bracket_4.0,income_bracket_5.0,income_bracket_6.0,income_bracket_7.0,income_bracket_8.0,income_bracket_9.0,income_bracket_10.0,income_bracket_11.0,income_bracket_12.0,income_bracket_Other
0,1,13,27,1053,0.0,train,X,19/05/13,05/07/13,46-55,...,0,1,0,0,0,0,0,0,0,0
1,2,13,116,48,0.0,train,X,19/05/13,05/07/13,36-45,...,0,0,0,0,0,0,0,0,0,0
2,6,9,635,205,0.0,train,Y,11/03/13,12/04/13,46-55,...,0,0,0,1,0,0,0,0,0,0
3,7,13,644,1050,0.0,train,X,19/05/13,05/07/13,Other,...,0,0,0,0,0,0,0,0,0,1
4,9,8,1017,1489,0.0,train,X,16/02/13,05/04/13,46-55,...,0,0,0,0,0,0,0,0,0,0


In [39]:
newtData.columns

Index([                         'id',                 'campaign_id',
                         'coupon_id',                 'customer_id',
                 'redemption_status',                      'dtType',
                     'campaign_type',                  'start_date',
                          'end_date',                   'age_range',
                    'marital_status',                      'rented',
                       'family_size',              'no_of_children',
                    'income_bracket',      'item_idtotalCountPurch',
            'item_iduniqueItemPurch',     'selling_pricetPCustomer',
       'selling_pricetPCustomerMean',  'selling_pricetPCustomerMin',
                       'quantitysum',        'other_discountoDCust',
          'other_discountoDCustMean',          'coupon_discountsum',
                            'patron',                             1,
                                   2,                             3,
                                  

In [ ]:
# customerProfileData.head(2)

In [ ]:
# customerProfileData.shape

In [40]:
newtData.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,campaign_type,start_date,end_date,age_range,...,income_bracket_4.0,income_bracket_5.0,income_bracket_6.0,income_bracket_7.0,income_bracket_8.0,income_bracket_9.0,income_bracket_10.0,income_bracket_11.0,income_bracket_12.0,income_bracket_Other
0,1,13,27,1053,0.0,train,X,19/05/13,05/07/13,46-55,...,0,1,0,0,0,0,0,0,0,0
1,2,13,116,48,0.0,train,X,19/05/13,05/07/13,36-45,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# newDataJoined=pd.merge(newtData,customerProfileData,left_on='customer_id',right_on='customer_id_',how='left')

In [ ]:
# newDataJoined.shape

In [ ]:
# newDataJoined.head(2)

In [ ]:
# newInfoData.head(2)

In [ ]:
# newCustUpdatedMainData=pd.merge(newDataJoined,newInfoData,on='customer_id',how='left')

In [41]:
newtData.shape

(128595, 78)

In [42]:
XVar=list(newtData.columns)
XVar.remove('id')
XVar.remove('redemption_status')
XVar.remove('start_date')
XVar.remove('end_date')
XVar.remove('dtType')
# XVar.remove('customer_id_')
YVar='redemption_status'

In [43]:
for j in categoColumns:
    XVar.remove(j)

In [44]:
trainPart=newtData[newtData['dtType']=='train']
testPart=newtData[newtData['dtType']=='test']

In [45]:
from sklearn import model_selection,metrics
import xgboost as xgb

In [46]:
XVar

['campaign_id',
 'coupon_id',
 'customer_id',
 'rented',
 'item_idtotalCountPurch',
 'item_iduniqueItemPurch',
 'selling_pricetPCustomer',
 'selling_pricetPCustomerMean',
 'selling_pricetPCustomerMin',
 'quantitysum',
 'other_discountoDCust',
 'other_discountoDCustMean',
 'coupon_discountsum',
 'patron',
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 26,
 27,
 28,
 29,
 30,
 'campaign_type_X',
 'campaign_type_Y',
 'age_range_18-25',
 'age_range_26-35',
 'age_range_36-45',
 'age_range_46-55',
 'age_range_56-70',
 'age_range_70+',
 'age_range_Other',
 'marital_status_Married',
 'marital_status_Other',
 'marital_status_Single',
 'family_size_1',
 'family_size_2',
 'family_size_3',
 'family_size_4',
 'family_size_5+',
 'family_size_Other',
 'no_of_children_0',
 'no_of_children_1',
 'no_of_children_2',
 'no_of_children_3+',
 'income_bracket_1.0',
 'income_bracket_2.0',
 'income_bracket_3.0',
 'income_bracket_4.0',
 'income_bracket_5.0',
 'income_bracket_6.0',
 'income_bracket_7.0'

In [47]:
trainX,valX,trainY,valY=model_selection.train_test_split(trainPart[XVar],trainPart[YVar],test_size=.3)
trainX.shape,valX.shape,trainY.shape,valY.shape

((54858, 67), (23511, 67), (54858,), (23511,))

In [49]:
trainX.head()

,campaign_id,coupon_id,customer_id,rented,item_idtotalCountPurch,item_iduniqueItemPurch,selling_pricetPCustomer,selling_pricetPCustomerMean,selling_pricetPCustomerMin,quantitysum,...,income_bracket_4.0,income_bracket_5.0,income_bracket_6.0,income_bracket_7.0,income_bracket_8.0,income_bracket_9.0,income_bracket_10.0,income_bracket_11.0,income_bracket_12.0,income_bracket_Other
27894,8,256,1245,0.0,839,328,83179.17,99.140846,0.71,233014,...,0,0,0,1,0,0,0,0,0,0
6582,5,689,1064,0.0,2118,950,295940.73,139.726501,6.77,291246,...,0,0,0,0,0,1,0,0,0,0
77103,13,420,858,0.0,989,542,81975.34,82.887098,7.48,13716,...,0,0,0,0,0,0,0,0,0,1
46764,13,909,9,0.0,557,405,57275.87,102.829210,7.12,117820,...,0,0,0,0,0,0,0,0,0,1
64035,7,641,865,0.0,1662,597,155596.98,93.620325,11.75,45174,...,0,0,0,1,0,0,0,0,0,0


In [50]:
modelR=xgb.XGBClassifier().fit(trainX,trainY)

In [51]:
trainScore2=modelR.predict(trainX)
trainScore=modelR.predict_proba(trainX)[:,1]
testScore=modelR.predict_proba(valX)[:,1]

In [52]:
sum(trainScore2)

3.0

In [53]:
sum(trainY)

507.0

In [54]:
metrics.roc_auc_score(trainY,trainScore)

0.9712593904831539

In [55]:
metrics.roc_auc_score(valY,testScore)

0.964634736501283

In [56]:
testFScore=modelR.predict_proba(testPart[XVar])[:,1]

In [57]:
len(testFScore)

50226

In [58]:
testPart.shape

(50226, 78)

In [59]:
test_QyjYwdjData.shape

(50226, 5)

In [60]:
firstSub=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:testFScore})

In [61]:
firstSub.head()

,id,redemption_status
0,3,0.015411
1,4,0.002645
2,5,0.075106
3,8,0.000243
4,10,0.000215


In [62]:
firstSub.to_csv('firstSub4.csv',index=False)

In [ ]:
print('train AUC score {} validation AUC score {}'.format())